In [111]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import pymongo
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk
import numpy as np

In [112]:
df = pd.read_csv('dataframe_finale.csv',delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            4200 non-null   object
 1   Artist           4200 non-null   object
 2   Rank             4200 non-null   int64 
 3   Last Week        4200 non-null   object
 4   Peak Positon     4200 non-null   int64 
 5   Weeks on charts  4200 non-null   int64 
 6   Date             4200 non-null   object
 7   Genre            4200 non-null   object
dtypes: int64(3), object(5)
memory usage: 262.6+ KB


In [113]:
df['Last Week'] = df['Last Week'].replace("-",0)
df['Last Week']= df['Last Week'].astype('int64')
df['Date']=pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Title            4200 non-null   object        
 1   Artist           4200 non-null   object        
 2   Rank             4200 non-null   int64         
 3   Last Week        4200 non-null   int64         
 4   Peak Positon     4200 non-null   int64         
 5   Weeks on charts  4200 non-null   int64         
 6   Date             4200 non-null   datetime64[ns]
 7   Genre            4200 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 262.6+ KB


In [114]:
data = df.to_dict('records')
data

[{'Title': 'Shut Down',
  'Artist': 'BLACKPINK',
  'Rank': 1,
  'Last Week': 0,
  'Peak Positon': 1,
  'Weeks on charts': 1,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': 'Pop, K-Pop (케이팝), Modern Classical, Orchestral, Girl Group, South Korea, Genius Korea'},
 {'Title': 'Pink Venom',
  'Artist': 'BLACKPINK',
  'Rank': 2,
  'Last Week': 4,
  'Peak Positon': 1,
  'Weeks on charts': 5,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': 'Pop, Korean, Hip-Hop, K-Pop (케이팝), Girl Group, South Korea, Genius Korea'},
 {'Title': "I'm Good (Blue)",
  'Artist': 'David Guetta & Bebe Rexha',
  'Rank': 3,
  'Last Week': 2,
  'Peak Positon': 2,
  'Weeks on charts': 4,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': 'Pop, Rap, Hip-Hop, Cloud Rap, Trap, Conscious Hip-Hop, Memes, West Coast'},
 {'Title': 'As It Was',
  'Artist': 'Harry Styles',
  'Rank': 4,
  'Last Week': 1,
  'Peak Positon': 1,
  'Weeks on charts': 25,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': 'Rock, Po

In [115]:
for song in data:
    song['Genre'] = song['Genre'].split(', ')
data

[{'Title': 'Shut Down',
  'Artist': 'BLACKPINK',
  'Rank': 1,
  'Last Week': 0,
  'Peak Positon': 1,
  'Weeks on charts': 1,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': ['Pop',
   'K-Pop (케이팝)',
   'Modern Classical',
   'Orchestral',
   'Girl Group',
   'South Korea',
   'Genius Korea']},
 {'Title': 'Pink Venom',
  'Artist': 'BLACKPINK',
  'Rank': 2,
  'Last Week': 4,
  'Peak Positon': 1,
  'Weeks on charts': 5,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': ['Pop',
   'Korean',
   'Hip-Hop',
   'K-Pop (케이팝)',
   'Girl Group',
   'South Korea',
   'Genius Korea']},
 {'Title': "I'm Good (Blue)",
  'Artist': 'David Guetta & Bebe Rexha',
  'Rank': 3,
  'Last Week': 2,
  'Peak Positon': 2,
  'Weeks on charts': 4,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': ['Pop',
   'Rap',
   'Hip-Hop',
   'Cloud Rap',
   'Trap',
   'Conscious Hip-Hop',
   'Memes',
   'West Coast']},
 {'Title': 'As It Was',
  'Artist': 'Harry Styles',
  'Rank': 4,
  'Last Week': 1,
  'Peak P

In [116]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,"scheme": "http"}])

In [117]:
es.ping()

True

In [118]:
def generate_data(data):
    for docu in data:
        yield {
            "_index": "billboard",
            "_type": "song",
            "_source": {k:v if v else None for k,v in docu.items()},
        }
        
if es.indices.exists('billboard')==True:
    es.indices.delete(index='billboard')
    bulk(es, generate_data(data))
else :
    bulk(es, generate_data(data))

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\2795846709.py:9: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists('billboard')==True:
c:\Users\camil\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [119]:
es.indices.exists('billboard')

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\1081182445.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.exists('billboard')


True

In [120]:
result = es.search(index="billboard", query= {"match_all": {}})
result

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4200, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'billboard',
    '_type': 'song',
    '_id': 'CbhmeoYBDgsOLAE7p2ng',
    '_score': 1.0,
    '_source': {'Title': 'Shut Down',
     'Artist': 'BLACKPINK',
     'Rank': 1,
     'Last Week': None,
     'Peak Positon': 1,
     'Weeks on charts': 1,
     'Date': '2022-10-01T00:00:00',
     'Genre': ['Pop',
      'K-Pop (케이팝)',
      'Modern Classical',
      'Orchestral',
      'Girl Group',
      'South Korea',
      'Genius Korea']}},
   {'_index': 'billboard',
    '_type': 'song',
    '_id': 'CrhmeoYBDgsOLAE7p2ng',
    '_score': 1.0,
    '_source': {'Title': 'Pink Venom',
     'Artist': 'BLACKPINK',
     'Rank': 2,
     'Last Week': 4,
     'Peak Positon': 1,
     'Weeks on charts': 5,
     'Date': '2022-10-01T00:00:00',
     'Genre': ['Pop',
      'Korean',
      'Hip-Hop',
      'K-Pop (케이팝

In [121]:
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

'4200 documents correspondent à la requêtes qui a pris 2 ms'

In [122]:
!curl http://localhost:9200/_cat/indices?v #!curl http://elasticsearch:9200/_cat/indices?v

health status index                    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .kibana_task_manager_1   EXIwN7TRSVqNquk2ayLb4A   1   0          2            1     16.8kb         16.8kb
green  open   .apm-agent-configuration UL5dxL77QXCnt2-BKIT2aQ   1   0          0            0       283b           283b
yellow open   billboard                eF_G4-JFRiGUfDOvAiy5Mg   1   1       4200            0    972.3kb        972.3kb
green  open   .kibana_1                jodusnl1QCuev0azrBTcxg   1   0         21            5     42.4kb         42.4kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   600  100   600    0     0  17467      0 --:--:-- --:--:-- --:--:-- 18181
curl: (3) URL using bad/illegal format or missing URL
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: elasticsearch


In [123]:
# cherche toutes les chansons avec pour artiste BLACKPINK
QUERY = {
    "query": {
        "term": {
            "Artist.keyword": "BLACKPINK"
        }
    }
}

In [124]:
result = es.search(index="billboard", body=QUERY)
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\3680423717.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'51 documents correspondent à la requêtes qui a pris 3 ms'

In [125]:
[elt['_source'] for elt in result["hits"]["hits"]]

[{'Title': 'Shut Down',
  'Artist': 'BLACKPINK',
  'Rank': 1,
  'Last Week': None,
  'Peak Positon': 1,
  'Weeks on charts': 1,
  'Date': '2022-10-01T00:00:00',
  'Genre': ['Pop',
   'K-Pop (케이팝)',
   'Modern Classical',
   'Orchestral',
   'Girl Group',
   'South Korea',
   'Genius Korea']},
 {'Title': 'Pink Venom',
  'Artist': 'BLACKPINK',
  'Rank': 2,
  'Last Week': 4,
  'Peak Positon': 1,
  'Weeks on charts': 5,
  'Date': '2022-10-01T00:00:00',
  'Genre': ['Pop',
   'Korean',
   'Hip-Hop',
   'K-Pop (케이팝)',
   'Girl Group',
   'South Korea',
   'Genius Korea']},
 {'Title': 'Typa Girl',
  'Artist': 'BLACKPINK',
  'Rank': 16,
  'Last Week': None,
  'Peak Positon': 16,
  'Weeks on charts': 1,
  'Date': '2022-10-01T00:00:00',
  'Genre': ['Pop',
   'K-Pop (케이팝)',
   'Hip-Hop',
   'Girl Group',
   'South Korea',
   'Genius Korea']},
 {'Title': 'Hard To Love',
  'Artist': 'BLACKPINK',
  'Rank': 27,
  'Last Week': None,
  'Peak Positon': 27,
  'Weeks on charts': 1,
  'Date': '2022-10-01T00

In [126]:
# affiche le titre et le rang pour le resultat de la requete
{elt['_source']['Title']:elt['_source']['Rank'] for elt in result["hits"]["hits"]}

{'Shut Down': 4,
 'Pink Venom': 10,
 'Typa Girl': 16,
 'Hard To Love': 27,
 'The Happiest Girl': 34,
 'Yeah Yeah Yeah': 43,
 'Tally': 52,
 'Ready For Love': 60}

In [127]:
# result['hits']['hits']
QUERY = {
    "query": {
        "term": {
            "Title.keyword": "Christmas"
        }
    }
}
result = es.search(index="billboard", body=QUERY)
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\3674638867.py:9: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'0 document correspondent à la requêtes qui a pris 0 ms'

In [128]:
# affiche les top 10 d'artistes qui ont le plus de chansons dans le billboard
QUERY ={
  "aggs": {
    "artist": {
      "terms": {
        "field": "Artist.keyword",
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["artist"]["buckets"]

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\1212785131.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key': 'Taylor Swift', 'doc_count': 150},
 {'key': 'Bad Bunny', 'doc_count': 102},
 {'key': 'Ed Sheeran', 'doc_count': 95},
 {'key': 'SZA', 'doc_count': 89},
 {'key': 'The Weeknd', 'doc_count': 79},
 {'key': 'Eminem', 'doc_count': 72},
 {'key': 'Harry Styles', 'doc_count': 59},
 {'key': 'Arctic Monkeys', 'doc_count': 56},
 {'key': 'BLACKPINK', 'doc_count': 51},
 {'key': 'Drake & 21 Savage', 'doc_count': 51}]

In [129]:
{
  "aggs": {
    "genres": {
      "terms": {
        "field": "genre.keyword"
      },
      "aggs": {
        "avg_rating": {
          "avg": {
            "field": "rating"
          }
        },
        "sales": {
          "sum": {
            "field": "sales"
          }
        }
      }
    }
  }
}


{'aggs': {'genres': {'terms': {'field': 'genre.keyword'},
   'aggs': {'avg_rating': {'avg': {'field': 'rating'}},
    'sales': {'sum': {'field': 'sales'}}}}}}

In [130]:
# chaque mois top des chansons qui apparaissent le plus sur le billbaord (pas en fonction de leur rang)
QUERY = {
  "aggs": {
    "song_by_month": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "month"
      },
      "aggs": {
        "title": {
          "terms": {
            "field": "Title.keyword"
          }
        }
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["song_by_month"]["buckets"]

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\3956439160.py:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key_as_string': '2022-10-01T00:00:00.000Z',
  'key': 1664582400000,
  'doc_count': 1000,
  'title': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 950,
   'buckets': [{'key': '2 Be Loved (Am I Ready)', 'doc_count': 5},
    {'key': '505', 'doc_count': 5},
    {'key': 'About Damn Time', 'doc_count': 5},
    {'key': 'After Like', 'doc_count': 5},
    {'key': "All Too Well (Taylor's Version)", 'doc_count': 5},
    {'key': 'Another Love', 'doc_count': 5},
    {'key': 'As It Was', 'doc_count': 5},
    {'key': 'Atlantis', 'doc_count': 5},
    {'key': 'B.O.T.A. (Baddest Of Them All)', 'doc_count': 5},
    {'key': 'Baby Shark', 'doc_count': 5}]}},
 {'key_as_string': '2022-11-01T00:00:00.000Z',
  'key': 1667260800000,
  'doc_count': 800,
  'title': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 755,
   'buckets': [{'key': 'Miss You', 'doc_count': 7},
    {'key': 'Die For You', 'doc_count': 6},
    {'key': '505', 'doc_count': 4},
    {'key': 'About Damn Time', 'doc_

In [131]:
QUERY= {
  "aggs": {
    "avg_weeks_on_chart": {
      "avg": {
        "field": "Weeks on charts"
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["avg_weeks_on_chart"]

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\1238742633.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


{'value': 31.349047619047617}

In [132]:
QUERY = {
  "aggs": {
    "top_song_per_month": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "month"
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["top_song_per_month"]

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\2806668549.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


{'buckets': [{'key_as_string': '2022-10-01T00:00:00.000Z',
   'key': 1664582400000,
   'doc_count': 1000},
  {'key_as_string': '2022-11-01T00:00:00.000Z',
   'key': 1667260800000,
   'doc_count': 800},
  {'key_as_string': '2022-12-01T00:00:00.000Z',
   'key': 1669852800000,
   'doc_count': 1000},
  {'key_as_string': '2023-01-01T00:00:00.000Z',
   'key': 1672531200000,
   'doc_count': 1000},
  {'key_as_string': '2023-02-01T00:00:00.000Z',
   'key': 1675209600000,
   'doc_count': 400}]}

In [133]:
# QUERY ={
#   "aggs": {
#     "title": {
#       "terms": {
#         "field": "Title.keyword"
#       },
#       "aggs": {
#         "artists": {
#           "terms": {
#             "field": "Artist.keyword"
#           }
#         }
#       }
#     }
#   }
# }
QUERY={
  "aggs": {
    "title": {
      "terms": {
        "field": "Title.keyword"
      },
      "aggs": {
        "artists": {
          "terms": {
            "field": "Artist.keyword"
          },
          "aggs": {
            "unique_artists": {
              "cardinality": {
                "field": "Artist.keyword"
              }
            }
          }
        }
      }
    }
  }
}

result = es.search(index="billboard", body=QUERY)
result = result["aggregations"]["title"]["buckets"]
result

C:\Users\camil\AppData\Local\Temp\ipykernel_20820\2297239962.py:41: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key': 'Die For You',
  'doc_count': 27,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'The Weeknd',
     'doc_count': 21,
     'unique_artists': {'value': 1}},
    {'key': 'Joji', 'doc_count': 6, 'unique_artists': {'value': 1}}]}},
 {'key': 'Miss You',
  'doc_count': 22,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Oliver Tree & Robin Schulz',
     'doc_count': 18,
     'unique_artists': {'value': 1}},
    {'key': 'southstar', 'doc_count': 4, 'unique_artists': {'value': 1}}]}},
 {'key': 'About Damn Time',
  'doc_count': 21,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Lizzo',
     'doc_count': 21,
     'unique_artists': {'value': 1}}]}},
 {'key': 'Another Love',
  'doc_count': 21,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Tom Odell',
     'doc_count': 21,
     '

In [134]:
for i in range(len(result)):
  title = result[i]['key'],result[i]['doc_count']
  print(title)
for i in range(len(result)):
  artistesdict = result[i]['artists']['buckets']
  for j in range(len(artistesdict)):
    artiste = artistesdict[j]['key'],artistesdict[j]['doc_count']
    print(artiste)

('Die For You', 27)
('Miss You', 22)
('About Damn Time', 21)
('Another Love', 21)
('As It Was', 21)
('Baby Shark', 21)
('Bad Habit', 21)
('Bad Habits', 21)
('Believer', 21)
('Blinding Lights', 21)
('The Weeknd', 21)
('Joji', 6)
('Oliver Tree & Robin Schulz', 18)
('southstar', 4)
('Lizzo', 21)
('Tom Odell', 21)
('Harry Styles', 21)
('Pinkfong', 21)
('Steve Lacy', 21)
('Ed Sheeran', 21)
('Imagine Dragons', 21)
('The Weeknd', 21)


In [135]:
df = pd.read_csv('my_data.csv')
df2 = pd.read_csv('my_data2.csv')


In [136]:
result = pd.merge(df, df2, on=["Artist", "Title"])

In [137]:
result

,Title,Artist,Release Date,Producers,Writers,Publisher,Distributors,Genre,Rank,Last Week,Peak Positon,Weeks on charts
0,Flowers,Miley Cyrus,"January 13, 2023",Kid Harpoon & Tyler Johnson,"Miley Cyrus, Aldae & Michael Pollack","Pure Tone Music, Suga Bear Recordz, Warner-Tam...",Nan,"Pop, Funk-Pop, Dance Rock, Pop-Rock, Nu Disco,...",1,1,1,4
1,Kill Bill,SZA,"December 9, 2022",Carter Lang & Rob Bisel,"SZA, Rob Bisel & Carter Lang",Nan,Sony Music Entertainment,"Pop, Alternative R&B, Alternative Pop, Psyched...",2,3,1,9
2,"Bzrp Music Sessions, Vol. 53",Bizarrap & Shakira,"January 11, 2023",Bizarrap,"Shakira, KEITYN, Santiago Alvarado & Bizarrap",Nan,The Orchard,"Pop, Deep House, Synth-Pop, Latin Urban, Latin...",3,2,2,5
3,Unholy,Sam Smith & Kim Petras,"September 22, 2022","Blake Slatkin, Cirkut, ILYA, Jimmy Napes, Omer...","Blake Slatkin, Cirkut, ILYA, Jimmy Napes, Kim ...",Nan,Nan,"R&B, Pop, UK Pop, Electronic, TikTok, Dark Pop...",4,4,1,20
4,Calm Down,Rema & Selena Gomez,"August 26, 2022",Andrevibez,"Andrevibez, Rema, Michael Hunter & Selena Gomez",Nan,Universal Music Group,"Pop, Remix, Nigerian Pidgin, Nigeria, Afrobeats",5,5,3,32
...,...,...,...,...,...,...,...,...,...,...,...,...
184,In The End,Linkin Park,"October 9, 2001",Don Gilmore,"Mr. Hahn, Rob Bourdon, Mike Shinoda, Chester B...",Nan,Nan,"Rap, Rock, Hard Rock, Alternative Metal, Rap M...",196,193,163,12
185,Ferrari,James Hype & Miggy Dela Rosa,"March 15, 2022",James Hype,"James Hype, Miggy Dela Rosa, Josh Grimmett, Jo...",Nan,Nan,"Pop, Deep House, House, Alternative Dance, Ele...",197,174,25,31
186,Just The Way You Are,Bruno Mars,"July 20, 2010",Needlz & The Smeezingtons,"Philip Lawrence, Ari Levine, Needlz, Saint Cas...",Nan,Nan,"R&B, Pop, Alternative R&B, Piano, Ballad, Adul...",198,-,162,12
187,Call Out My Name,The Weeknd,"March 30, 2018",Ging,"Ging, Nicolas Jaar & The Weeknd","Alma Road Music, 4AD Music, Beggars Music, Win...",Nan,"Pop, R&B, Singer-Songwriter, Downtempo, Canada...",199,181,97,31


In [141]:
result['Publisher']

0      Pure Tone Music, Suga Bear Recordz, Warner-Tam...
1                                                    Nan
2                                                    Nan
3                                                    Nan
4                                                    Nan
5                                                    Nan
6      BLACKSTONE CREATIVE, Pulse Music Group & Conco...
7                                                    Nan
8                                                    Nan
9      Ducky Donath Music, Universal Music Group, Son...
10                                                   Nan
11                                                   Nan
12     MJP Music, Amaya Sofia Publishing & Universal ...
13                                                   Nan
14                      Sony Music Entertainment & ARESA
15     La Leyenda Publishing, Universal Music Group &...
16                                                   Nan
17                             